# Wrangling for VisualSYNTH

Steps:

* First are *manual* transformations, they will use the currenct selection to suggest a single transformation.
* Second, we can suggest transformations for other parts of the spreadsheet as well, by considering multiple arguments.
* Third, we can suggest combinations of transformations by using the synth-a-sizer algorithm.
* Finally, we add support for colorings to be used.


In [69]:
import pandas as pd

class WranglingTest:
    
    def __init__(self):
        self.dataframes = {}
        self.selected_headers = {}
        self.generateTestDataframes()

    def generateTestDataframes(self):
        self.dataframes["sales"] = pd.read_csv("data/sales.csv")
        self.dataframes["prod"] = pd.read_csv("data/prod.csv")
        self.selected_headers["sales"] = set()
        self.selected_headers["prod"] = set()
        self.selected_headers["sales"].add("Type")

    def run(self):
        print(self.dataframes["sales"])
        print(self.dataframes["prod"])


w = WranglingTest()
w.run()

        Type Country   June   July  August   Total Profit
0     Banana      BE  170.0  690.0   520.0  1380.0    YES
1        NaN      DE  610.0  640.0   320.0  1570.0     NO
2        NaN      DE  250.0  650.0   630.0  1530.0    YES
3  Chocolate      BE  560.0  320.0   140.0  1020.0    YES
4        NaN      FR  430.0  350.0   300.0  1080.0    YES
5        NaN     NaN    NaN    NaN     NaN     NaN    NaN
6        NaN      NL  210.0  280.0   270.0   760.0     NO
7  Speculaas      BE  300.0  270.0   290.0   860.0     NO
8    Vanilla      BE  610.0  190.0   670.0  1470.0    YES
        Type  ProdTime
0  Chocolate        60
1     Banana        40
2  Speculaas        70
3    Vanilla        40


# Manual

Based on selection, apply some transformation.

## Fill

In [70]:
class FillTransformation(WranglingTest):
    
    def is_available(self):  
        for name, columns in self.selected_headers.items():
            for column in columns:
                if self.dataframes[name][column].isnull().any():
                    return True
        return False
    
    def run(self):
        result = {name: df.copy() for name, df in self.dataframes.items()}
        for name, columns in self.selected_headers.items():
            for column in columns:
                if result[name][column].isnull().any():
                    result[name][column] = result[name][column].ffill()
        return result

ft = FillTransformation()
ft.is_available()

True

In [71]:
ft.run()

{'sales':         Type Country   June   July  August   Total Profit
 0     Banana      BE  170.0  690.0   520.0  1380.0    YES
 1     Banana      DE  610.0  640.0   320.0  1570.0     NO
 2     Banana      DE  250.0  650.0   630.0  1530.0    YES
 3  Chocolate      BE  560.0  320.0   140.0  1020.0    YES
 4  Chocolate      FR  430.0  350.0   300.0  1080.0    YES
 5  Chocolate     NaN    NaN    NaN     NaN     NaN    NaN
 6  Chocolate      NL  210.0  280.0   270.0   760.0     NO
 7  Speculaas      BE  300.0  270.0   290.0   860.0     NO
 8    Vanilla      BE  610.0  190.0   670.0  1470.0    YES,
 'prod':         Type  ProdTime
 0  Chocolate        60
 1     Banana        40
 2  Speculaas        70
 3    Vanilla        40}

## Drop

Drops rows in which the selected column has an empty value.

In [67]:
class DropTransformation(WranglingTest):
    def __init__(self):
        super().__init__()

        self.selected_headers = {"sales": {"Country"}, "prod": set()}

    def is_available(self):
        for name, columns in self.selected_headers.items():
            for column in columns:
                if self.dataframes[name][column].isnull().any():
                    return True
        return False

    def run(self):
        result = {name: df.copy() for name, df in self.dataframes.items()}
        for name, columns in self.selected_headers.items():
            for column in columns:
                df = result[name]
                em = df[column].isnull()
                result[name] = df[~em]
        return result


dt = DropTransformation()
dt.is_available()

True

In [68]:
for _, t in dt.run().items():
    display(t)

,Type,Country,June,July,August,Total,Profit
0,Banana,BE,170.0,690.0,520.0,1380.0,YES
1,NaN,DE,610.0,640.0,320.0,1570.0,NO
2,NaN,DE,250.0,650.0,630.0,1530.0,YES
3,Chocolate,BE,560.0,320.0,140.0,1020.0,YES
4,NaN,FR,430.0,350.0,300.0,1080.0,YES
6,NaN,NL,210.0,280.0,270.0,760.0,NO
7,Speculaas,BE,300.0,270.0,290.0,860.0,NO
8,Vanilla,BE,610.0,190.0,670.0,1470.0,YES


,Type,ProdTime
0,Chocolate,60
1,Banana,40
2,Speculaas,70
3,Vanilla,40


## Fold


In [65]:
class FoldTransformation(WranglingTest):
    def __init__(self):
        super().__init__()
        self.selected_headers = {
            "sales": {"June", "July", "August"},
            "prod": set(),
        }

    def is_available(self):
        """Selected consecutive columns."""
        for name, columns in self.selected_headers.items():
            if len(columns) > 0:
                dfc = self.dataframes[name].columns.tolist()
                ixs = sorted(dfc.index(c) for c in columns)
                if ixs == list(range(min(ixs), max(ixs) + 1)):
                    return True
        return False

    def run(self):
        result = {name: df.copy() for name, df in self.dataframes.items()}
        for name, columns in self.selected_headers.items():
            if len(columns) > 0:
                dfc = self.dataframes[name].columns.tolist()
                ixs = [dfc.index(c) for c in columns]
                if sorted(ixs) == list(range(min(ixs), max(ixs) + 1)):
                    result[name] = pd.melt(
                        result[name],
                        value_vars=columns,
                        id_vars=set(dfc) - columns,
                    )
        return result


ft = FoldTransformation()
ft.is_available()

True

In [66]:
for _, t in ft.run().items():
    display(t)

,Profit,Type,Total,Country,variable,value
0,YES,Banana,1380.0,BE,July,690.0
1,NO,NaN,1570.0,DE,July,640.0
2,YES,NaN,1530.0,DE,July,650.0
3,YES,Chocolate,1020.0,BE,July,320.0
4,YES,NaN,1080.0,FR,July,350.0
5,NaN,NaN,NaN,NaN,July,NaN
6,NO,NaN,760.0,NL,July,280.0
7,NO,Speculaas,860.0,BE,July,270.0
8,YES,Vanilla,1470.0,BE,July,190.0
9,YES,Banana,1380.0,BE,August,520.0


,Type,ProdTime
0,Chocolate,60
1,Banana,40
2,Speculaas,70
3,Vanilla,40
